In [1]:
from pygraphblas import *
from pygraphblas import lib
from pygraphblas.types import Type, binop
from pygraphblas.gviz import draw, draw_vis, draw_vector, draw_matrix
import numpy as np
import pprint
from pyvis import network as net

import this
options_set(nthreads=8)

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:


class GS(Type):
    
    _base_name = "UDT"
    _numpy_t = None
    members = ['double w', 'uint64_t n']
    one = (float('inf'), 0)
    
    @binop(boolean=True) 
    def EQ(z, x, y):
        if x.w == y.w and x.n == y.n:
            z = True
        else:
            z = False

    @binop()
    def PLUS(z, x, y):
        if x.w < y.w:
            z.w = x.w
            z.n = x.n
        elif x.w == y.w:
            z.w = x.w
            z.n = x.n + y.n
        else:
            z.w = y.w
            z.n = y.n

    @binop()
    def TIMES(z, x, y):
        z.w = x.w + y.w
        z.n = x.n * y.n

GS_monoid = GS.new_monoid(GS.PLUS, GS.one)
GS_semiring = GS.new_semiring(GS_monoid, GS.TIMES)

def shortest_path(matrix, start):
    n = matrix.nrows
    v = Vector.sparse(matrix.type, n)
    for i, j, k in matrix:
        if i == j:
            matrix[i,j] = (float('inf'), 0)
        else:
            matrix[i,j] = (k[0], 1)        
    v = matrix.extract_row(start)
    v_k = matrix.extract_row(start)
    with GS_semiring:
        for _ in range(matrix.nrows+1):
            v_k = v_k @ matrix
            v = v + v_k
    return v

A = Matrix.sparse(GS, 6, 6)
A[0,1] = (9.0, 1)
A[0,3] = (3.0, 1)
A[0,5] = (4.0, 1)
A[1,2] = (8.0, 1)
A[3,4] = (6.0, 1)
A[3,5] = (1.0, 1)
A[4,2] = (4.0, 1)
A[1,5] = (7.0, 1)
A[5,4] = (2.0, 1)

N = net.Network(notebook=True, directed=True)
for i, j, v in A:
    N.add_node(i, label=str(i), shape='circle')
    N.add_node(j, label=str(j), shape='circle')
    N.add_edge(i, j, label=str(v[0]))
N.show('graph.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [3]:
g = draw(A, label_vector=shortest_path(A, 0))

In [4]:
def shortest_path_FW(matrix):
    n = matrix.nrows
    v = Vector.sparse(matrix.type, n)
    for i, j, k in matrix:
        if i == j:
            matrix[i,j] = (float('inf'), 0)
        else:
            matrix[i,j] = (k[0], 1)        
    D_k = matrix.dup()
    D = matrix.dup()
    with GS_semiring:
        for _ in range(matrix.nrows):
            D_k @= matrix
            D += D_k
    return D

In [5]:
D = shortest_path_FW(A)

In [6]:
for i, j, k in D:
    print(i, j, D[i, j])

0 1 (9.0, 1)
0 2 (10.0, 2)
0 3 (3.0, 1)
0 4 (6.0, 2)
0 5 (4.0, 2)
1 2 (8.0, 1)
1 4 (9.0, 1)
1 5 (7.0, 1)
3 2 (7.0, 1)
3 4 (3.0, 1)
3 5 (1.0, 1)
4 2 (4.0, 1)
5 2 (6.0, 1)
5 4 (2.0, 1)


In [7]:
N = net.Network(notebook=True, directed=True, height=300)
labels = {}
for i, j, v in D:
    if j not in labels:
        labels[j] = [f"({i},{j},{v[0]},{v[1]})"]
    else:
        labels[j].append(f"({i},{j},{v[0]},{v[1]})")
for i, j, v in A:
    if i in labels:
        N.add_node(i, label='\n'.join(labels[i]), shape='circle')
    else:
        N.add_node(i, label=str(i), shape='circle')
    if j in labels:
        N.add_node(j, label='\n'.join(labels[j]), shape='circle')
    else:
        N.add_node(j, label=str(j), shape='circle')
    N.add_edge(i, j, label=str(v))     
N.show('graph.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [8]:
def algebraic_PC(Adj, states):
    n = Adj.shape[0]
    A = Matrix.sparse(GS, n, n)
    S = 0.0
    S_vt = dict()
    S_sv = dict()
    for v in range(n):
        deltas = states - states[v]
        S_vt[v] = np.sum(deltas[deltas > 0])
        deltas = states * (-1) + states[v]
        S_sv[v] = np.sum(deltas[deltas > 0])
        S += S_vt[v]
    for k, v in Adj.items():
        i = k[0]
        j = k[1]
        if i == j:
            A[i,j] = (float('inf'), 0)
        else:
            A[i,j] = (v, 1)
    D = shortest_path_FW(A)
    PC = Vector.sparse(FP32, n)  
    for v in range(n):
        S_exclude_v = S - S_sv[v] - S_vt[v]
        for s in D.extract_col(v).indices:
            for t in D.extract_row(v).indices:
                if s != v and t!=v and s!=t and D[s, t][0] == D[s, v][0] + D[v, t][0]:
                    delta = states[t] - states[s]
                    if delta <= 0:
                        continue
                    w = delta / S_exclude_v 
                    if v in PC.indices:
                        PC[v] += D[s, v][1] * D[v, t][1] / D[s, t][1] * w
                    else:
                        PC[v] = D[s, v][1] * D[v, t][1] / D[s, t][1] * w
    return PC

In [9]:
from scipy.sparse import dok_matrix

Adj = dok_matrix((6, 6), dtype=float) 
Adj[0,1] = 9.0
Adj[0,3] = 3.0
Adj[0,5] = 4.0
Adj[1,2] = 8.0
Adj[3,4] = 6.0
Adj[3,5] = 1.0
Adj[4,2] = 4.0
Adj[1,5] = 7.0
Adj[5,4] = 2.0

states = np.array([0, 0, 0.4, 0, 0, 0])
PC = algebraic_PC(Adj, states)
print(PC)

0|
1|
2|
3|0.125
4|0.75
5|0.5


In [10]:
def algebraic_PC_with_paths(Adj, states):
    n = Adj.shape[0]
    A = Matrix.sparse(GS, n, n)
    A1 = Matrix.sparse(FP32, n, n)
    S = 0.0
    S_vt = dict()
    S_sv = dict()
    for v in range(n):
        deltas = states - states[v]
        S_vt[v] = np.sum(deltas[deltas > 0])
        deltas = states * (-1) + states[v]
        S_sv[v] = np.sum(deltas[deltas > 0])
        S += S_vt[v]
    for k, v in Adj.items():
        i = k[0]
        j = k[1]
        if i == j:
            A[i,j] = (float('inf'), 0)
        else:
            A[i,j] = (v, 1)
            A1[i,j] = v
    D = shortest_path_FW(A)
    D1 = Matrix.sparse(FP32, n, n)
    for i, j, v in D:
        D1[i,j] = v[0]
    print(A1)
    print(D1)
    pred = dict()
    with FP32.plus:
        for i in range(n):
            pred[i] = dict()
            d = D1.extract_row(i)
            d[i] = 0
            for j in d.indices:
                col_j = A1.extract_col(j)
                col_j.emult(d, mult_op=FP32.plus, out=col_j)
                #pred_j = list((col_j == d[j]).indices)
                #if pred_j:
                pred[i][j] = list((col_j == d[j]).indices)
    PC = Vector.sparse(FP32, n)
    paths = dict()
    v_paths = dict()
    for v in range(n):
        S_exclude_v = S - S_sv[v] - S_vt[v]
        v_paths[v] = []
        for s in D.extract_col(v).indices:
            for t in D.extract_row(v).indices:
                if s != v and t!=v and s!=t and D[s, t][0] == D[s, v][0] + D[v, t][0]:
                    delta = states[t] - states[s]
                    if delta <= 0:
                        continue
                    w = delta / S_exclude_v
                    if (s, t) not in paths:
                        paths[(s,t)] = st_paths(pred[s], s, t)
                    for p in paths[(s, t)]:
                        if v in p:
                            v_paths[v].append(p)
                    if v in PC.indices:
                        PC[v] += D[s, v][1] * D[v, t][1] / D[s, t][1] * w
                    else:
                        PC[v] = D[s, v][1] * D[v, t][1] / D[s, t][1] * w
    return PC, v_paths, pred

def st_paths(pred, s, t):
    stack = [[t, 0]]
    top = 0
    paths = []
    while top >= 0:
        node, i = stack[top]
        if node == s:
            paths.append([p for p, n in reversed(stack[:top+1])])
        if node not in pred:
            continue
        if len(pred[node]) > i:
            top += 1
            if top == len(stack):
                stack.append([pred[node][i],0])
            else:
                stack[top] = [pred[node][i],0]
        else:
            stack[top-1][1] += 1
            top -= 1
    return paths
    
            

In [11]:
PC, paths, pred = algebraic_PC_with_paths(Adj, states)
print(PC)
print(pred)

      0  1  2  3  4  5
  0|   9.0   3.0   4.0|  0
  1|      8.0      7.0|  1
  2|                  |  2
  3|            6.01.0|  3
  4|      4.0         |  4
  5|            2.0   |  5
      0  1  2  3  4  5
      0  1  2  3  4  5
  0|   9.010.03.06.04.0|  0
  1|      8.0   9.07.0|  1
  2|                  |  2
  3|      7.0   3.01.0|  3
  4|      4.0         |  4
  5|      6.0   2.0   |  5
      0  1  2  3  4  5
0|
1|
2|
3|0.125
4|0.75
5|0.5
{0: {0: [], 1: [0], 2: [4], 3: [0], 4: [5], 5: [0, 3]}, 1: {1: [], 2: [1], 4: [5], 5: [1]}, 2: {2: []}, 3: {2: [4], 3: [], 4: [5], 5: [3]}, 4: {2: [4], 4: []}, 5: {2: [4], 4: [5], 5: []}}


In [12]:
v = 0
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [13]:
v = 1
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [14]:
v = 2
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [15]:
v = 3
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [16]:
v = 4
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [17]:
v = 5
if v in pred:
    N = net.Network(notebook=True, directed=True, height=300)
    for k, val in Adj.items():
        i = k[0]
        j = k[1]
        N.add_node(i, label=str(i), shape='circle')
        N.add_node(j, label=str(j), shape='circle')
        if j not in pred[v]:
            continue
        if i in pred[v][j]:
            N.add_edge(i, j, label=str(val))     
N.show('sssp.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
